In [192]:
import requests
import json
import pandas as pd
import numpy as np
import ta

In [2]:
#Daily Timeframe Data Grabber
def daily_df(ticker):

    API_URL = "https://www.alphavantage.co/query"

    data = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": "compact",
        "datatype": "json",
        "apikey": "PVGTC9P6IUS7XLQ9",
        }

    response = requests.get(API_URL, params=data)
    
    if response.status_code == 200:
        stock_data = response.json()
        meta_data = stock_data['Meta Data']
        daily_data = stock_data['Time Series (Daily)']
        daily_dataframe_rev = pd.DataFrame.from_dict(daily_data)
        daily_dataframe = daily_dataframe_rev.transpose()     
        daily_dataframe.columns = ['open', 'high', 'low', 'close', 'volume']
        df_entries2float(daily_dataframe)
        return daily_dataframe

In [3]:
#Converting dataframe entries to floats
def df_entries2float(df):
    for i in df.columns:
        column2convert = df[i]
        column_size = column2convert.size
        for k in range(0,column_size):
            column2convert[k] = float(column2convert[k])
    return df

In [4]:
#Reversing dataframe
def rev(df):
    return df.sort_index(ascending = True, axis = 0)

In [5]:
#Running TA with proper formatting
def tech_indicator(indicator, daily_df):
    indicator_df = pd.DataFrame(indicator)
    indicator_df = indicator_df.sort_index(ascending=False, axis=0)
    resulting_df = daily_df.join(indicator_df)
    return resulting_df

In [54]:
#All technical indicators in use
def run_indicators(df):
    high = rev(df['high'])
    low = rev(df['low'])    
    close = rev(df['close'])
    volume = rev(df['volume'])
    
    df = tech_indicator(ta.momentum.wr(high,low,close),df)
    df = tech_indicator(ta.momentum.money_flow_index(high,low,close,volume),df)
    df = tech_indicator(ta.momentum.stoch_signal(high,low,close),df)
    df = tech_indicator(ta.momentum.tsi(close),df)
    df = tech_indicator(ta.trend.macd(close),df)
    df = tech_indicator(ta.trend.trix(close),df)
    df = tech_indicator(ta.trend.aroon_up(close),df)
    df = tech_indicator(ta.trend.aroon_down(close),df)
    df = tech_indicator(ta.volatility.keltner_channel_hband_indicator(high,low,close),df)
    df = tech_indicator(ta.volatility.keltner_channel_lband_indicator(high,low,close),df)
    df = tech_indicator((ta.volatility.bollinger_hband_indicator(close)),df)
    df = tech_indicator((ta.volatility.bollinger_lband_indicator(close)),df)  
    df = tech_indicator(ta.volume.chaikin_money_flow(high,low,close,volume),df)
    
    df['bbi'] = df['bbihband'].subtract(df['bbilband'])
    df['kci'] = df['kci_hband'].subtract(df['kci_lband'])
    del df['bbihband']
    del df['bbilband']
    del df['kci_hband']
    del df['kci_lband']
    df = tech_indicator(ta.momentum.rsi(close),df)
    
    return df

In [205]:
#Mapping values
def map_values(df):
    length = len(df)
    
    df = df.copy()

    #############
    ####TREND####
    #############
    
    #Mapping MACD values
    macd = df.columns.get_loc('MACD_12_26')
    for i in range(length):
        if 0 < df.iloc[i,macd]:
            df.iloc[i,macd] = 1
        elif df.iloc[i,macd] < 0:
            df.iloc[i,macd] = -1
        else:
            df.iloc[i,macd] = 0
            
    #Mapping TRIX values
    trix = df.columns.get_loc('trix_15')
    for i in range(length):
        if 0 < df.iloc[i,trix]:
            df.iloc[i,trix] = 1
        elif df.iloc[i,trix] < 0:
            df.iloc[i,trix] = -1
        else:
            df.iloc[i,trix] = 0
            
    #Mapping Aroon values
    values = []
    aroon_up = df.columns.get_loc('aroon_up25')
    aroon_down = df.columns.get_loc('aroon_down25')
    for i in range(length):
        if df.iloc[i,aroon_up] >= 70 and df.iloc[i,aroon_down] <= 30:
            values.append(2.0)
        elif df.iloc[i,aroon_down] >= 70 and df.iloc[i,aroon_up] <= 30:
            values.append(-2.0)
        elif df.iloc[i,aroon_up] > df.iloc[i,aroon_down]:
            values.append(1.0)
        elif df.iloc[i,aroon_down] > df.iloc[i,aroon_up]:
            values.append(-1.0)
        else:
            values.append(0.0)
    df['aroon'] = values
    
    #Mapping Chaikin values
    cmf = df.columns.get_loc('cmf')
    for i in range(length):
        if 0.05 <= df.iloc[i,cmf] < 0.1:
            df.iloc[i,cmf] = 1
        elif -0.1 < df.iloc[i,cmf] <= -0.05:
            df.iloc[i,cmf] = -1
        elif df.iloc[i,cmf] >= 0.1:
            df.iloc[i,cmf] = 2
        elif df.iloc[i,cmf] <= -0.1:
            df.iloc[i,cmf] = -2
        else:
            df.iloc[i,cmf] = 0
            
    #############
    #OSCILLATORS#
    #############
    
    #Mapping MFI values
    mfi = df.columns.get_loc('mfi_14')
    for i in range(length):
        if 0 <= df.iloc[i,mfi] <= 20:
            df.iloc[i,mfi] = 2
        elif 80 <= df.iloc[i,mfi] <= 100:
            df.iloc[i,mfi] = -2
        elif 20 <= df.iloc[i,mfi] < 30:
            df.iloc[i,mfi] = 1
        elif 70 < df.iloc[i,mfi] <= 80:
            df.iloc[i,mfi] = -1
        else:
            df.iloc[i,mfi] = 0
            
    #Mapping rsi values
    rsi = df.columns.get_loc('rsi')
    for i in range(length):
        if 70 <= df.iloc[i,rsi] < 100:
            df.iloc[i,rsi] = -2
        elif 0 <= df.iloc[i,rsi] <= 30:
            df.iloc[i,rsi] = 2
        elif 30 < df.iloc[i,rsi] <= 40:
            df.iloc[i,rsi] = 1
        elif 60 <= df.iloc[i,rsi] < 70:
            df.iloc[i,rsi] = -1
        else:
            df.iloc[i,rsi] = 0

    #Mapping Stoch_d values
    stoch = df.columns.get_loc('stoch_d')
    for i in range(length):
        if 80 <= df.iloc[i,stoch] <= 100:
            df.iloc[i,stoch] = -2
        elif 0 <= df.iloc[i,stoch] <= 20:
            df.iloc[i,stoch] = 2
        elif 20 < df.iloc[i,stoch] <= 30:
            df.iloc[i,stoch] = 1
        elif 70 <= df.iloc[i,stoch] < 80:
            df.iloc[i,stoch] = -1
        else:
            df.iloc[i,stoch] = 0
            
    #Mapping TSI values
    tsi = df.columns.get_loc('tsi')
    for i in range(length):
        if df.iloc[i,tsi] < -25:
            df.iloc[i,tsi] = -2
        elif df.iloc[i,tsi] <= -10:
            df.iloc[i,tsi] = -1
        elif df.iloc[i,tsi] <= 10:
            df.iloc[i,tsi] = 0
        elif df.iloc[i,tsi] < 25:
            df.iloc[i,tsi] = 1
        else:
            df.iloc[i,tsi] = 2
            
    #Mapping wr values
    wr = df.columns.get_loc('wr')
    for i in range(length):
        if -100 <= df.iloc[i,wr] <= -80:
            df.iloc[i,wr] = 2
        elif -20<= df.iloc[i,wr] <= 0:
            df.iloc[i,wr] = -2
        elif -30 <= df.iloc[i,wr] < -20:
            df.iloc[i,wr] = -1
        elif -80 < df.iloc[i,wr] <= -70:
            df.iloc[i,wr] = 1
        else:
            df.iloc[i,wr] = 0
            
    del df['aroon_up25']
    del df['aroon_down25']
    
    df['Overall Rating'] = (df['wr'] + df['mfi_14'] + df['stoch_d'] + 
                          df['tsi'] + df['MACD_12_26'] + df['trix_15'] + 
                          df['cmf'] + df['bbi'] + df['kci'] + df['rsi'] +
                          df['aroon'])
    
    return df

In [84]:
#Predicting stock based on the ticker
def stock_predict(ticker):
    df = daily_df(ticker)
    df = run_indicators(df)
    df = map_values(df)
    df = df[:-45]
    return df #Returns indicator signals and consensus

In [312]:
#Returning mapped indicator data less OHLCV
def indicator_data(ticker):
    df = daily_df(ticker)
    df = run_indicators(df)
    df = map_values(df)
    df = df[:-45]
    df = df.rename(columns = {'wr':'William\'s PR','mfi_14':'Money Flow Index','stoch_d':'Stochastic Oscilattor',
                              'tsi':'True Strength Index','MACD_12_26':'MACD','trix_15':'Trix','cmf':'Chaikin Money Flow',
                              'bbi':'Bollinger Bands','kci':'Keltner Channel','rsi':'Relative Strength Index',
                              'aroon':'Aroon'})
    del df['open']
    del df['close']
    del df['volume']
    del df['low']
    del df['high']
    df = df.astype(float)
    df = df.astype(str)
    for i in range(len(test)):
        for k in range(0, 11):
            if df.iloc[i][k] == str(-2.0):
                df.iloc[i][k] = "Strong Sell"
            elif df.iloc[i][k] == str(-1.0):
                df.iloc[i][k] = "Sell"
            elif df.iloc[i][k] == str(0.0):
                df.iloc[i][k] = "Neutral"
            elif df.iloc[i][k] == str(1.0):
                df.iloc[i][k] = "Buy"
            elif df.iloc[i][k] == str(2.0):
                df.iloc[i][k] = "Strong Buy"  
    return df #Returns indicator signals and consensus

In [94]:
#Predicting stock based on the DataFrame input
def stock_predict_HC(df):
    df = run_indicators(df)
    df = map_values(df)
    df = df[:-45]
    return df #Returns indicator signals and consensus

In [200]:
#Creating dataframe of signals for all constituents
constituents = pd.read_csv('constituents_csv.csv')
ticker = constituents['Symbol'][0]
raw_data = pd.read_csv("Stock Data/" + ticker + ".csv", index_col=0) 
predicted_data_raw = stock_predict_HC(raw_data)
predicted_data_preprocessed = pd.DataFrame(predicted_data_raw['Overall Rating'])
predicted_data_abs_final = predicted_data_preprocessed.rename(columns = {'Overall Rating':ticker})

for i in range(1, len(constituents)):
    try:
        ticker = constituents['Symbol'][i]
        raw_data = pd.read_csv("Stock Data/" + ticker + ".csv", index_col=0) 
        predicted_data_raw = stock_predict_HC(raw_data)
        predicted_data_preprocessed = pd.DataFrame(predicted_data_raw['Overall Rating'])
        predicted_data_final = predicted_data_preprocessed.rename(columns = {'Overall Rating':ticker})
        predicted_data_abs_final = predicted_data_abs_final.join(predicted_data_final)      
    except:
        continue

predicted_data_abs_final = predicted_data_abs_final.sort_values(by=[predicted_data_abs_final.index[0]], axis = 1, ascending = False)
predicted_data_abs_final = predicted_data_abs_final.dropna(axis = 1)

In [ ]:
data_averaged = predicted_data_abs_final.copy() 
data_averaged.head(10)

In [ ]:
ao10 = [] 

for i in range(len(data_averaged.columns)):
    
    sum = 0
    
    for j in range(10):
        sum += data_averaged.iloc[j][i]
        
    average = sum/10
    ao10.append(average)
    
data_averaged = pd.DataFrame(np.array([ao10]), columns=data_averaged.columns).append(data_averaged, ignore_index=False)
data_averaged = data_averaged.rename(index={0: 'average of 10'})
data_averaged.head()

In [ ]:
data_averaged = data_averaged.sort_values(by=[data_averaged.index[0]], axis = 1, ascending = False)
data_averaged.head(5)

In [253]:
def top20(df, position='buy'):
    if position == 'buy':
        top20_buy = pd.DataFrame(df.iloc[0,:20])
        top20_buy = top20_buy.reset_index()
        top20_buy = top20_buy.rename(columns={'index':'Ticker','average of 10':'Score'})
        top20_buy.index += 1
        return top20_buy
    if position == 'sell':
        top20_sell = pd.DataFrame(df.iloc[0,-20:])
        top20_sell = top20_sell.rename(columns={'index':'Ticker','average of 10':'Score'})
        top20_sell = top20_sell.sort_values(by=['Score'],axis=0,ascending=True)
        top20_sell = top20_sell.reset_index()
        top20_sell.index += 1
        top20_sell = top20_sell.rename(columns={'index':'Ticker'})
        return top20_sell

In [256]:
top20(data_averaged, 'sell')

,Ticker,Score
1,HCA,-8.7
2,FCX,-8.0
3,CCL,-7.9
4,DVA,-7.6
5,MS,-7.5
6,FDX,-7.5
7,AXP,-7.5
8,SCHW,-7.4
9,SLB,-7.4
10,MMM,-7.4
